In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
metadata_path = "../metadata"

In [8]:
# Create a dictionary of DataFrames, one for each language
lang_data = {
    lang_dir: pd.concat(
        [pd.read_csv(f'{metadata_path}/{lang_dir}/cv-corpus-15.0-2023-09-08/{lang_dir}/{set_name}.tsv', delimiter="\t")
         for set_name in ["train", "test", "validated"]],
    )
    for lang_dir in os.listdir(metadata_path)
}

C:\Users\anais\AppData\Local\Temp\ipykernel_12824\2757715039.py:4: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  [pd.read_csv(f'{metadata_path}/{lang_dir}/cv-corpus-15.0-2023-09-08/{lang_dir}/{set_name}.tsv', delimiter="\t")
C:\Users\anais\AppData\Local\Temp\ipykernel_12824\2757715039.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  [pd.read_csv(f'{metadata_path}/{lang_dir}/cv-corpus-15.0-2023-09-08/{lang_dir}/{set_name}.tsv', delimiter="\t")
C:\Users\anais\AppData\Local\Temp\ipykernel_12824\2757715039.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  [pd.read_csv(f'{metadata_path}/{lang_dir}/cv-corpus-15.0-2023-09-08/{lang_dir}/{set_name}.tsv', delimiter="\t")
C:\Users\anais\AppData\Local\Temp\ipykernel_12824\2757715039.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low

In [9]:
#Print the number of speakers for each language
for lang,df in lang_data.items():
    print(lang, df["client_id"].nunique())

ar 1066
de 17667
en 77437
es 16927
fr 16848
id 365
it 6688
ja 1462
lg 592
ru 2661
sw 1080
ta 520
tr 1425
zh-CN 5379


In [ ]:
#Print the total duration for each language
#TODO

In [16]:
languages = [lang for lang in os.listdir(metadata_path)]
n_speakers = np.zeros(len(languages))

for set_name in ["train","test","validated"]:
    n_speakers+=np.array([pd.read_csv(train_path,sep="\t")\
        ["client_id"]\
        .nunique() 
        for train_path in Path(metadata_path).rglob(f"{set_name}.tsv")])

df = pd.DataFrame({"language" : languages, "n_speakers" : n_speakers})
df

C:\Users\anais\AppData\Local\Temp\ipykernel_12364\4061150121.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  n_speakers+=np.array([pd.read_csv(train_path,sep="\t")\
C:\Users\anais\AppData\Local\Temp\ipykernel_12364\4061150121.py:5: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  n_speakers+=np.array([pd.read_csv(train_path,sep="\t")\
C:\Users\anais\AppData\Local\Temp\ipykernel_12364\4061150121.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  n_speakers+=np.array([pd.read_csv(train_path,sep="\t")\
C:\Users\anais\AppData\Local\Temp\ipykernel_12364\4061150121.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  n_speakers+=np.array([pd.read_csv(train_path,sep="\t")\
C:\Users\anais\AppData\Local\Temp\ipykernel_12364\4061150121.py:5: DtypeWarning: Columns (7,8) 

,language,n_speakers
0,ar,2048.0
1,de,29944.0
2,en,125298.0
3,es,27917.0
4,fr,29001.0
5,id,697.0
6,it,11786.0
7,ja,2535.0
8,lg,1094.0
9,ru,4805.0


In [17]:
durations = [pd.read_csv(cd, sep="\t")["duration[ms]"]\
    .sum()
    for cd in Path(metadata_path).rglob("clip_durations.tsv")]

df['total_duration[ms]']=durations
df

,language,n_speakers,total_duration[ms]
0,ar,2048.0,553629078
1,de,29944.0,4994673077
2,en,125298.0,12047305099
3,es,27917.0,7873329654
4,fr,29001.0,3902596354
5,id,697.0,229541988
6,it,11786.0,1378725780
7,ja,2535.0,813315453
8,lg,1094.0,2011923063
9,ru,4805.0,932842296
